In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [3]:
# Import the Sequential model and layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding = 'same'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

batch_size = 5

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 298, 298, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      18496     
_________________________________________________________________
activation_1 (Activation)    (None, 147, 147, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 128)       7

In [4]:
# Training Augmentation configuration
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Testing Augmentation - Only Rescaling
test_datagen = ImageDataGenerator(rescale = 1./255)

# Generates batches of Augmented Image data
#this is a generator that will read pictures found in C:\\Users\\allen\\train and indefinitely generate 
#batches of augmented image data
train_generator = train_datagen.flow_from_directory('C:\\Users\\allen\\train',  # importing images
                                                    target_size = (300, 300), # all images will be resized to 300x300
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical') 

# Generator for validation data
validation_generator = test_datagen.flow_from_directory('C:\\Users\\allen\\test', # importing images
                                                        target_size = (300, 300),
                                                        batch_size = batch_size,
                                                        class_mode = 'categorical')


Found 723 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [5]:
# Fit the model on Training data
model.fit_generator(train_generator,
                    epochs = 20,
                    validation_data = validation_generator,
                    verbose = 1)

# Evaluating model performance on Testing data
loss, accuracy = model.evaluate(validation_generator)

print("\nModel's Evaluation Metrics: ")
print("---------------------------")
print("Accuracy: {} \nLoss: {}".format(accuracy, loss))

C:\Users\allen\anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
145/145 [==============================] - 95s 638ms/step - loss: 0.8946 - accuracy: 0.5740 - val_loss: 0.6010 - val_accuracy: 0.6500
Epoch 2/20
145/145 [==============================] - 95s 655ms/step - loss: 0.6388 - accuracy: 0.6625 - val_loss: 0.4178 - val_accuracy: 0.8167
Epoch 3/20
145/145 [==============================] - 91s 626ms/step - loss: 0.5462 - accuracy: 0.7261 - val_loss: 0.4798 - val_accuracy: 0.7167
Epoch 4/20
145/145 [==============================] - 91s 627ms/step - loss: 0.4819 - accuracy: 0.7566 - val_loss: 0.7658 - val_accuracy: 0.7167
Epoch 5/20
145/145 [==============================] - 90s 623ms/step - loss: 0.4403 - accuracy: 0.7732 - val_loss: 0.3599 - val_accuracy: 0.8500
Epoch 6/20
145/145 [==============================] - 92s 631ms/step - loss: 0.6004 - accuracy: 0.7303 - val_loss: 0.6408 - val_accuracy: 0.7000
Epoch 7/20
145/145 [==============================] - 91s 625ms/step - loss: 0.4855 - accuracy: 0.7801 - val_loss: 0.4228 - val_ac

Here we can see that the model has an accuracy of 83.33% and loss of 0.34. Therefore the model is performing well.

In [15]:
model.save_weights('first_try_image.h5') # saving the model

In [6]:
# predicting the class of images using the trained model
predict_generator = ImageDataGenerator(rescale = 1./255)

predict_generator = predict_generator.flow_from_directory('C:\\Users\\allen\\predict', 
                                                        target_size = (300, 300),
                                                        batch_size = batch_size,
                                                        class_mode = 'categorical')

Found 90 images belonging to 1 classes.


In [7]:
predictions = model.predict(predict_generator)

In [10]:
import pandas as pd

In [11]:
Predictions = pd.DataFrame(predictions, columns=['Jeans', 'Saree','Trouser'])

In [12]:
Predictions # predicted data

,Jeans,Saree,Trouser
0,9.094689e-01,0.048458,0.999477
1,9.999417e-01,0.001092,0.999627
2,7.693771e-01,0.003275,0.999968
3,9.988963e-01,0.017502,0.999119
4,9.998775e-01,0.000278,0.997930
...,...,...,...
85,9.619828e-01,0.353145,0.966463
86,9.999788e-01,0.000127,0.999689
87,3.542349e-01,0.163881,0.994196
88,8.179051e-07,1.000000,0.482292
